In [2]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [3]:
# Read in csv
df = pd.read_csv("output_data/arpa_projects_jan_22_to_jan_23.csv")

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (0,1,2,3,4,5,6,7,14,15,16,18,23,24,25,26,27,28,29,31,32,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### law enforcement
police, PD, gun, law enforcement, public safety, crime, criminal, body cameras, tasers, armor, sheriff, officer

### courts
court, public defenders, prosecutors, juvenile court, juvenile justice

### corrections: 
jail, prison, correction, incarcerated, inmate, guards, custody


In [4]:
list_le = ["police","PD","gun","law enforcement","public safety","crime","criminal","body cameras","tasers","armor","sheriff","officer"]

list_court = ["court","public defenders","prosecutors","juvenile court","juvenile justice"]

list_correction = ["jail","prison","correction","incarcerated","inmate","guards","custody"]

In [5]:
df["law_enforcement"] = df["Project Description"].str.lower().str.contains("|".join(list_le))== True
df["court"] = df["Project Description"].str.lower().str.contains("|".join(list_court))== True
df["corrections"] = df["Project Description"].str.lower().str.contains("|".join(list_correction))== True

In [6]:
df.columns

Index(['Recipient Name', 'State/Territory', 'Reporting Tier', 'Recipient Type', 'Project Name', 'Expenditure Category Group', 'Expenditure Category', 'Project Description', 'Adopted Budget', 'Total Obligations', 'Total Expenditures', 'Number of Households Served (Select ECs Only)', 'Number of Workers Served (Select ECs Only)', 'Number of Gov FTE Supported', 'Community benefit agreement? (Infra Only)', 'Complying with David Bacon? (Infra Only)', 'Project labor agreement? (Infra Only)', 'dataset', 'Recipient-ID', 'Total Cumulative Obligations', 'Total Cumulative Expenditures', 'Number of Households Served (Select Expenditure Categories Only)', 'Number of Workers Served (Select Expenditure Categories Only)', 'Number of Gov FTE Responding to COVID-19 (Select Expenditure Categories Only)', 'Number of Gov FTE Rehired (Select Expenditure Categories Only)', 'Community benefit agreement? (Infrastructure Only)', 'Complying with David Bacon? (Infrastructure Only)',
       'Project labor agreement

In [7]:
def find_cj_projects(row):
    if row["law_enforcement"] | row["court"] | row["corrections"]:
        return True
    else:
        pass

In [8]:
df["cj_related"] = df.apply(find_cj_projects, axis = 1)

In [9]:
df.to_csv("output_data/cj_related_projects_to_vet.csv")

In [10]:
# df = pd.read_csv("source_data/cj_related_projects_vetted.csv")

In [11]:
df_cj_related_by_state = df[df["cj_related"] == True]\
    .groupby("State/Territory")\
    ["Total Obligations"].sum().reset_index()\
    .rename(columns = {
    "Total Obligations": "cj_related_obligations"
})

In [12]:
df_all_by_state = df\
    .groupby("State/Territory")\
    ["Total Obligations"].sum().reset_index()\
    .rename(columns = {
    "Total Obligations": "all_obligations"
})

In [13]:
df_state_merge = pd.merge(df_cj_related_by_state, df_all_by_state, on="State/Territory", how="right")

In [14]:
df_state_merge["cj_share"] = df_state_merge["cj_related_obligations"]/df_state_merge["all_obligations"]

In [15]:
df_state_merge.sort_values("cj_related_obligations", ascending = False)

,State/Territory,cj_related_obligations,all_obligations,cj_share
5,California,1.076723e+10,1.684945e+10,0.639026
41,Pennsylvania,4.541661e+09,4.906234e+09,0.925692
34,New York,1.009421e+09,3.181550e+09,0.317273
47,Texas,2.881322e+08,8.247265e+09,0.034937
38,Ohio,1.740292e+08,1.934863e+09,0.089944
15,Illinois,1.465770e+08,8.478534e+08,0.172880
10,Florida,1.376726e+08,1.352948e+09,0.101757
11,Georgia,1.360157e+08,5.706400e+08,0.238356
13,Hawaii,1.006037e+08,1.319179e+09,0.076262
1,Alaska,9.428317e+07,2.772224e+08,0.340099


In [16]:
df["Recipient Name"].nunique()

15096

In [18]:
df_cj_related_by_recipient = df[df["cj_related"] == True]\
    .groupby("Recipient Name")\
    ["Total Obligations"].sum().reset_index()\
    .rename(columns = {
    "Total Obligations": "cj_related_obligations"
})

df_all_by_recipient = df\
    .groupby("Recipient Name")\
    ["Total Obligations"].sum().reset_index()\
    .rename(columns = {
    "Total Obligations": "all_obligations"
})

df_recipient_merge = pd.merge(df_cj_related_by_recipient, df_all_by_recipient, on="Recipient Name", how="right")

df_recipient_merge["cj_share"] = df_recipient_merge["cj_related_obligations"]/df_recipient_merge["all_obligations"]

In [19]:
df_recipient_merge.sort_values("cj_related_obligations", ascending = False).head(20)

,Recipient Name,cj_related_obligations,all_obligations,cj_share
10715,State of California,9.196313e+09,1.344999e+10,0.683741
3635,Commonwealth Of Pennsylvania,4.491006e+09,4.491006e+09,1.000000
2373,"City Of New York, New York",9.201425e+08,2.451956e+09,0.375269
7162,"Los Angeles City, California",3.173652e+08,3.473652e+08,0.913636
2268,"City And County Of San Francisco, California",3.124087e+08,3.124087e+08,1.000000
7119,"Long Beach City, California",1.246971e+08,1.357531e+08,0.918558
2389,"City Of Sacramento, California",1.123133e+08,1.123133e+08,1.000000
3463,"Cleveland, Ohio",1.089605e+08,1.139869e+08,0.955904
197,"Anaheim City, California",1.066302e+08,1.066302e+08,1.000000
10717,State of Hawaii,1.005985e+08,1.297958e+09,0.077505


In [20]:
df_state_merge.to_csv("output_data/state_merge_not_vetted.csv", index=False)
df_recipient_merge.to_csv("output_data/recipient_merge_not_vetted.csv", index=False)

In [84]:

# conditions = [(df['Project Description'].str.lower().str.contains('police')== True),
#               (df['Project Description'].str.contains('PD')== True),
#               (df['Project Description'].str.lower().str.contains('gun')== True),
#               (df['Project Description'].str.lower().str.contains('law enforcement')== True), 
#               (df['Project Description'].str.lower().str.contains('public safety')== True),
#               (df['Project Description'].str.lower().str.contains('crime')== True),
#               (df['Project Description'].str.lower().str.contains('criminal')== True),
#               (df['Project Description'].str.lower().str.contains('body cameras')== True),
#               (df['Project Description'].str.lower().str.contains('tasers')== True),
#               (df['Project Description'].str.lower().str.contains('armor')== True),
#               (df['Project Description'].str.lower().str.contains('sheriff')== True),
#               (df['Project Description'].str.lower().str.contains('officer')== True),
#               (df['Project Description'].str.lower().str.contains('jail')== True),
#               (df['Project Description'].str.lower().str.contains('prison')== True),
#               (df['Project Description'].str.lower().str.contains('correction')== True),
#               (df['Project Description'].str.lower().str.contains('incarcerated')== True),
#               (df['Project Description'].str.lower().str.contains('inmate')== True),
#               (df['Project Description'].str.lower().str.contains('juvenile court')== True),
#               (df['Project Description'].str.lower().str.contains('juvenile justice')== True),
#               (df['Project Description'].str.lower().str.contains('custody')== True)]

# choices = ["police", "police", "le", "le", "le", "le", "le" ,"equipment", "equipment", "equipment", "staff", "staff", "doc", "doc", "doc", "doc", "doc", "doc", "doc", "doc"]
# df["le_cat"] = np.select(conditions, choices, default="other")
# df["law_enforcement"] = np.where((df["le_cat"] != "other"), 1, 0)

# recipients = df.groupby(["SLT Application ID", 'Recipient Name', "State/Territory", "law_enforcement", "Total Expenditures"])["Total Obligations"].sum().reset_index()
# recipients_le = recipients[recipients["law_enforcement"]!=0].sort_values(by=['Total Obligations'], ascending = False)

# result = pd.merge(recipients_le, tranche, on=["SLT Application ID","Recipient Name"])
# result = result.rename(columns={result.columns[-1]: 'tranche'})

# result['share_le']=result['Total Expenditures']/result['tranche']
# result.sort_values(by='share_le', ascending = False)

# result.to_csv("output_data/arpa_le.csv")
